# 2. Your first API request

In this section we're going to learn how to send a request for information to the Trove API.

API requests are just like normal urls. However, instead of sending us back a web page, they deliver data in a form that computers can understand. We can then use that data in our own programs.

We're going to use the Python [Requests](http://docs.python-requests.org/en/master/) library to handle our API queries, so let's import it now.

In [3]:
import requests

## Getting an API key
Any requests you make to the Trove API need to be authenticated with a 'key'. For non-commercial projects, you just  fill out a simple form and your API key is generated instantly. Follow the instructions in the Trove Help to [obtain your own Trove API Key](http://help.nla.gov.au/trove/building-with-trove/api).

Once you've created a key, you can access it at any time on the 'For developers' tab of your Trove user profile.

Copy your API key now, and paste it in the cell below, between the quotes.

In [4]:
api_key = '6pi5hht0d2umqcro'
print('Your API key is: {}'.format(api_key))

Your API key is: 6pi5hht0d2umqcro


All search queries to the Trove API start with the same base url. We'll save it as a variable here.

In [5]:
api_search_url = 'https://api.trove.nla.gov.au/result'

Trove API queries are constructed by adding parameters to the base url. Most of the parameters are optional, but a few are mandatory:

* `q` &ndash; 'q' for query, this is where search terms go
* `zone` &ndash; which Trove zone (or zones) do you want to search, use 'all' for everything
* `key` &ndash; your Trove API key

If you don't want to specify a search term, you can just use a space or a plus sign &ndash; '&nbsp;' or '+' &ndash; as the value for `q`. Of course, this means that you're asking for *everything*, so Trove might take a bit longer to respond.

The default output of the API is XML. For most applications it's easier to work with JSON. You set this using the `encoding` parameter.

We'll meet some other parameters later, but for now let's create a Python dictionary to store our basic parameters. The `requests` library will take this dictionary, turn it into a string, and add it to the base url.


In [6]:
params = {
    'q': 'cyclone', # Search for this keyword -- feel free to change!
    'zone': 'newspaper', # Search in the newspaper zone
    'key': api_key,
    'encoding': 'json'
}

Ok, we're now now ready to make our first query!

In [7]:
response = requests.get(api_search_url, params=params)
print(response.url) # This shows us the url that's sent to the API

https://api.trove.nla.gov.au/result?q=cyclone&zone=newspaper&key=6pi5hht0d2umqcro&encoding=json


The `response` variable contains the results in json format.

In [ ]:
data = response.json()

In [16]:
# Let's prettify the raw JSON data before displaying it
formatted_data = json.dumps(data, indent=2)
highlighted_data = highlight(formatted_data, lexers.JsonLexer(), formatters.TerminalFormatter())
print(highlighted_data)

{
  "response": {
    "query": "cyclone",
    "zone": [
      {
        "name": "newspaper",
        "records": {
          "s": "0",
          "n": "20",
          "total": "591135",
          "next": "/result?q=cyclone&encoding=json&zone=newspaper&s=20",
          "article": [
            {
              "id": "214555312",
              "url": "/newspaper/214555312",
              "heading": "CYCLONE",
              "category": "Article",
              "title": {
                "id": "1175",
                "value": "Hamersley News (Perth, WA: 1969 - 1987)"
              },
              "date": "1977-11-10",
              "page": 2,
              "pageSequence": 2,
              "relevance": {
                "score": "10.133797",
                "value": "very relevant"
              },
              "snippet": "... <strong>CYCLONE</strong>                     ...",
              "troveUrl": "http://trove.nla.gov.au/ndp/del/article/214555312?searchTerm=cyclone"
            },
    

The individual item records are quite deeply nested in the results.

`data['response']['zone']` is a list of dictionaries, one dictionary for each zone.

Each dictionary includes the following values:

* `name` -- the name of the zone
* `records` -- another dictionary containing the search results
    
The `records` dictionary includes the following values:

* `total` -- the number of matching search results in this zone
* `n` -- the number of results in this set
* `s` -- the starting point of this result set
* `next` -- a path which when added to the base API url will return the next set of results
* and of course the results set itself

Somewhat annoyingly the key for the results set is different in different zones, so you need to know what it is in advance.
 
| Zone       | Key       |
| ---------- | --------- |
| list       | `list`    |
| people     | `people`  |
| newspaper  | `article` |
| book       | `work`    |
| article    | `work`    |
| picture    | `work`    |
| map        | `work`    |
| collection | `work`    |
| music      | `work`    |

For example, to get the results set from the newspapers you'd use `records['article']`.

In [ ]:
def zone_totals(data):
    '''
    Display totals in each zone.
    '''
    zones = []
    for zone in data['response']['zone']:
        print('{:<15} {:,}'.format(zone['name'], int(zone['records']['total'])))
        zones.append(zone['name'])

zone_totals(data)

While the API returns results for the `people` zone, it's not really returning much useful data. Hopefully this will be fixed in the future, but for now you need to use a separate `people` API.

In [ ]:
record_labels = {
    'list': 'list',
    'people': 'people',
    'newspaper': 'article',
    'book': 'work',
    'article': 'work',
    'picture': 'work',
    'map': 'work',
    'collection': 'work',
    'music': 'work'
}

In [ ]:
def list_titles(zone_name):
    '''
    List the titles in a specified zone.
    eg: list_titles('newspaper')
    '''
    for zone in data['response']['zone']:
        if zone['name'] == zone_name:
            for record in zone['records'][record_labels[zone_name]]:
                if 'heading' in record:
                    print(record['heading'])
                elif 'title' in record:
                    print(record['title'])

In [ ]:
list_titles('book')